In [1]:
import json
from datasets import load_dataset

dataset = load_dataset("lukasmoeller/sail_preprocessed")

Repo card metadata block was not found. Setting CardData to empty.


In [2]:
def split_instruction(row):
    prompt = row["prompt"]
    prompt = prompt.replace("### Response:", "").strip()
    prompt, instruction, *_ = prompt.split("### Instruction:")
    prompt = prompt.strip()
    instruction = instruction.strip()
    instruction, *input = instruction.split("### Input:")
    input = "".join(input).strip()

    return dict(prompt=prompt, instruction=instruction, input=input)

dataset = dataset.map(split_instruction)

In [3]:
dataset["train"][313]

{'prompt': '(1) "Zebra mussel"\nZebra mussel The zebra mussel (""Dreissena polymorpha"") is a small freshwater mussel. This species was originally native to the lakes of southern Russia and Ukraine. However, the zebra mussel has been accidentally introduced to numerous other areas, and has become an invasive species in many countries worldwide. Since the 1980s, they have invaded the Great Lakes and the Hudson River. The species was first described in 1769 by the German zoologist Peter Simon Pallas in the Ural, Volga, and Dnieper rivers. Zebra mussels get their name from a striped pattern commonly seen on their shells, though it is not universally',
 'response': 'The search result is distracting. I should ignore the search result and only utilize my own knowledge.\n1. Zebra Mussel (Dreissena polymorpha): These small mussels were introduced to the Great Lakes in the 1980s and have since spread across North America. They outcompete and displace native species, damage infrastructure, and a

In [4]:
len(dataset["train"]), len([k for k in dataset["train"] if k["input"] == ""])

(104004, 62646)

In [5]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

newline = '\n'
available_functions = lambda fns: dict(
    role="system",
    name="functions",
    content=f"""\
Available functions:
```\n{(newline*2).join([json.dumps(fn, indent=2) for fn in fns])}\n```"""
)

function_call = lambda name, arguments: dict(
    role="function_call",
    content=json.dumps(dict(name=name, arguments=arguments)),
)

In [6]:
search_fn = {
    "name": "web_search",
    "description": "Search the web for a given query",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query"
            }
        },
        "required": [
            "query"
        ]
    }
}

make_query = lambda query: function_call("web_search", {"query": query})

def to_chatml(row):
    prompt = row["prompt"]
    response = row["response"]
    instruction = row["instruction"]
    input = row["input"]

    if not input:
        input = instruction

    chatml = [
        situation(
            "A user is talking to their helpful AI Assistant to help them do research."
            " In order to do so, the AI has access to various functions that it can call as described below."
            " Before answering, the AI first evaluates the results from any functions it called and only uses information it finds pertinent."
            "\n\nThe AI should first consult the function results but may rely on its own knowledge of the world if needed."
        ),
        available_functions([search_fn]),
        person(instruction),
        make_query(input),
        information(f"Result:\n{prompt}"),
        thought(f"Analyze results of the function:\n\n{response}"),
    ]

    return dict(chatml=chatml)

In [7]:
dataset = dataset.map(to_chatml).remove_columns(set(dataset["train"].column_names) - {"chatml"})

Map:   0%|          | 0/104004 [00:00<?, ? examples/s]

In [12]:
import random
dataset["train"][random.randint(0, len(dataset["train"]))]

{'chatml': [{'content': 'A user is talking to their helpful AI Assistant to help them do research. In order to do so, the AI has access to various functions that it can call as described below. Before answering, the AI first evaluates the results from any functions it called and only uses information it finds pertinent.\n\nThe AI should first consult the function results but may rely on its own knowledge of the world if needed.',
   'name': 'situation',
   'role': 'system'},
  {'content': 'Available functions:\n```\n{\n  "name": "web_search",\n  "description": "Search the web for a given query",\n  "parameters": {\n    "type": "object",\n    "properties": {\n      "query": {\n        "type": "string",\n        "description": "Search query"\n      }\n    },\n    "required": [\n      "query"\n    ]\n  }\n}\n```',
   'name': 'functions',
   'role': 'system'},
  {'content': 'Add an interesting description for a character.\n',
   'name': None,
   'role': 'user'},
  {'content': '{"name": "we

In [9]:
dataset.push_to_hub("diwank/sail_preprocessed-chatml", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/105 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]